In [1]:
cd D:\Production\Biocytin 10x

D:\Production\Biocytin 10x


In [2]:
import os, sys
import glob as glob
from pathlib import Path
import requests
import shutil

In [3]:
#make list of all of the .vsi files (images) in the handed off directory (the only directory whose images we check for deletability
paths = glob.glob("Handed off/*.vsi")

#for each path string, parse the 10 digit barcode from the string and add it to the barcode list
barcodes_to_check = [path[11:21] for path in paths]
barcodes_to_check

['0400271776',
 '0400274736',
 '0500405276',
 '0535190472',
 '2250331010',
 '2250331019',
 '2250331020',
 '2250331026',
 '2250414629',
 '2250414644',
 '2250414645',
 '2250421008',
 '2250421024',
 '2250428026',
 '2250428027',
 '2250505365',
 '2250505366',
 '2250505367',
 '2250505369',
 '2250505370',
 '2250505371',
 '2250505372',
 '2250505373',
 '2250505374',
 '2250505375',
 '2250505376',
 '2250505377',
 '2250505378',
 '2250505385',
 '2250505386',
 '2250505387',
 '2250509813',
 '2250509814',
 '2250509815',
 '2250509816',
 '2250509817',
 '2250509818',
 '2250509819',
 '2250509820',
 '2250509821',
 '2250509822',
 '2250509823',
 '2250509824',
 '2250509825',
 '2250509826',
 '2250509827',
 '2250509828',
 '2250509829',
 '2250509830',
 '2250509832',
 '2250516490',
 '2250516492',
 '2250523812',
 '2250523813',
 '2250523814',
 '2250523815',
 '2250523816',
 '2250523817',
 '2503101003',
 '2503170012',
 '2503170032']

In [4]:
#make empty list of slides that haven't handed off properly (for later reporting on handoff issues)
error_barcodes_and_statuses = {}

#for every barcode found in the 'handed off' directory:
for barcode in barcodes_to_check:
    #this 'params' syntax isn't super familiar to me, but the URL I based this on ended in '?barcode=2408154435' so I deleted that before
    #passing it as the positional argument and... it worked...
    request = requests.get('https://lims2.corp.alleninstitute.org/slides/info/details.json', params={"barcode": barcode})
    
    #if the LIMS request yields an ok code
    if request.status_code == 200:
        
        #assign the json file to this variable
        lims_info = request.json()
        
        #from that json (functions like a dictionary?) assign the value from the workflow_state key to this variable
        workflow_state = lims_info["workflow_state"]
        
        #print that state
        print(f"{barcode}'s workflow state is {workflow_state}")

        #if the workflow state is passed or qc, delete the vsi. delete the images folder, and all of its contents.
        if workflow_state == "passed" or workflow_state == "qc":
            try:    
                shutil.rmtree(f"handed off/_{barcode}_")
                os.remove(f"handed off/{barcode}.vsi")
                
            except Exception as e:
                print(e)
            #print(glob.glob(f"handed off/*{barcode}*"))
            
        #if it wasn't in state 'passed' or 'qc', add the barcode and state as a key: value pair in the dictionary
        #this dictionary gets printed after the loop concludes
        else:
            error_barcodes_and_statuses[f'{barcode}'] = workflow_state
    else:
        print(print(f"{barcode}'s get request failed"))
        error_barcodes_and_statuses[f'{barcode}'] = 'get request failed'

print(f'Errors:\n{error_barcodes_and_statuses}')

0400271776's workflow state is pending
0400274736's workflow state is passed
0500405276's workflow state is passed
0535190472's workflow state is passed
2250331010's workflow state is passed
2250331019's workflow state is pending
2250331020's workflow state is pending
2250331026's workflow state is passed
2250414629's workflow state is passed
2250414644's workflow state is pending
2250414645's workflow state is pending
2250421008's workflow state is pending
2250421024's workflow state is pending
2250428026's workflow state is pending
2250428027's workflow state is pending
2250505365's workflow state is passed
2250505366's workflow state is passed
2250505367's workflow state is passed
2250505369's workflow state is passed
2250505370's workflow state is passed
2250505371's workflow state is passed
2250505372's workflow state is passed
2250505373's workflow state is passed
2250505374's workflow state is passed
2250505375's workflow state is passed
2250505376's workflow state is passed
225

In [5]:
for i in error_barcodes_and_statuses:
    print(i)

0400271776
2250331019
2250331020
2250414644
2250414645
2250421008
2250421024
2250428026
2250428027
2503101003
2503170012
2503170032


In [ ]:
# edits to script: be able to parse pending slides based on whether or not they have scans in LIMS. pending barcodes with scanners
# eg VS200FLUOR, VS200BRIGHT, should be ready to delete because the image was already uploaded

In [6]:
def lims_check(barcodes_to_check):
    """checks LIMS status via GET request, returns dictionary of barcodes and statuses as keys and values"""
    barcode_statuses = {}
    barcodes_to_check = [path[11:21] for path in paths]
    for barcode in barcodes_to_check:
        #this 'params' syntax isn't super familiar to me, but the URL I based this on ended in '?barcode=2408154435' so I deleted that and... it worked...
        request = requests.get('https://lims2.corp.alleninstitute.org/slides/info/details.json', params={"barcode": barcode})
        if request.status_code == 200:
            lims_info = request.json()
            workflow_state = lims_info["workflow_state"]
            barcode_statuses[barcode] = workflow_state
    return barcode_statuses

In [7]:
lims_check(paths)

{'0400239866': 'passed',
 '0400269804': 'passed',
 '0400270747': 'passed',
 '0400270931': 'passed',
 '0400273584': 'passed',
 '0400273952': 'passed',
 '0400275319': 'passed',
 '0400275823': 'passed',
 '0500404856': 'passed',
 '0500405114': 'passed',
 '0500406541': 'passed',
 '0500407946': 'passed',
 '0500410869': 'passed',
 '0500411809': 'passed',
 '0500411977': 'passed',
 '0500415520': 'passed',
 '0500416106': 'passed',
 '0500419327': 'passed',
 '0500419853': 'passed',
 '0500420464': 'passed',
 '0500420498': 'passed',
 '0535187165': 'passed',
 '0535189859': 'passed',
 '2503101001': 'passed',
 '2503101002': 'passed',
 '2503101003': 'pending',
 '2503101004': 'passed',
 '2503101005': 'passed',
 '2503101006': 'passed',
 '2503101007': 'passed',
 '2503101008': 'passed',
 '2503101009': 'passed',
 '2503101010': 'passed',
 '2503101011': 'passed',
 '2503101012': 'passed',
 '2503101013': 'pending',
 '2503101014': 'pending',
 '2503101015': 'pending',
 '2503101016': 'passed',
 '2503101017': 'pendi

In [9]:
p =Path.home()
p

WindowsPath('C:/Users/ics')

In [10]:
p.joinpath("herpderp")

WindowsPath('C:/Users/ics/herpderp')

In [16]:
def files_and_folders_statuses(directory):
    """checks LIMS status via GET request, returns dictionary of barcodes and statuses as keys and values"""
    barcode_statuses = {}
    paths = directory.glob("*.vsi")
    #barcodes_to_check = [path.stem for path in paths]
    for filename in paths:
        barcode = filename.stem
        #this 'params' syntax isn't super familiar to me, but the URL I based this on ended in '?barcode=2408154435' so I deleted that and... it worked...
        request = requests.get('https://lims2.corp.alleninstitute.org/slides/info/details.json', params={"barcode": barcode})
        if request.status_code == 200:
            lims_info = request.json()
            workflow_state = lims_info["workflow_state"]
            barcode_statuses[filename] = workflow_state
            foldername = filename.parent.joinpath(f"_{barcode}_")
            if foldername.exists():
                barcode_statuses[foldername] = workflow_state
                
    return barcode_statuses

In [19]:
status_dict = files_and_folders_statuses(Path("Handed off"))

0it [00:00, ?it/s]

In [27]:
def delete_or_not(dictionary, dryrun=True):
    """deletes a item or appends it to a dictionary of non-passed barcodes"""
    error_barcodes_and_statuses = {}
    for barcode, workflow_state in dictionary.items():
        if workflow_state == 'passed':
            if dryrun:
                print(glob.glob(f"handed off/*{barcode}*"))
            else:
                 try:    
                    shutil.rmtree(f"handed off/_{barcode}_")
                    os.remove(f"handed off/{barcode}.vsi")
                 except Exception as e:
                    print(e)
                
        else:
            error_barcodes_and_statuses[f'{barcode}'] = workflow_state
    return(error_barcodes_and_statuses)
        

In [29]:
delete_or_not(status_dict, dryrun=False)

[WinError 3] The system cannot find the path specified: 'handed off/_Handed off\\0400239866.vsi_'
[WinError 3] The system cannot find the path specified: 'handed off/_Handed off\\_0400239866__'
[WinError 3] The system cannot find the path specified: 'handed off/_Handed off\\0400269804.vsi_'
[WinError 3] The system cannot find the path specified: 'handed off/_Handed off\\_0400269804__'
[WinError 3] The system cannot find the path specified: 'handed off/_Handed off\\0400270747.vsi_'
[WinError 3] The system cannot find the path specified: 'handed off/_Handed off\\_0400270747__'
[WinError 3] The system cannot find the path specified: 'handed off/_Handed off\\0400270931.vsi_'
[WinError 3] The system cannot find the path specified: 'handed off/_Handed off\\_0400270931__'
[WinError 3] The system cannot find the path specified: 'handed off/_Handed off\\0400273584.vsi_'
[WinError 3] The system cannot find the path specified: 'handed off/_Handed off\\_0400273584__'
[WinError 3] The system cannot

{'Handed off\\2503101003.vsi': 'pending',
 'Handed off\\_2503101003_': 'pending',
 'Handed off\\2503101013.vsi': 'pending',
 'Handed off\\_2503101013_': 'pending',
 'Handed off\\2503101014.vsi': 'pending',
 'Handed off\\_2503101014_': 'pending',
 'Handed off\\2503101015.vsi': 'pending',
 'Handed off\\_2503101015_': 'pending',
 'Handed off\\2503101017.vsi': 'pending',
 'Handed off\\_2503101017_': 'pending',
 'Handed off\\2503101029.vsi': 'pending',
 'Handed off\\_2503101029_': 'pending'}